# Sensitivity analysis of the critical head difference

In this example, we will demonstrate how to conduct a sensitivity analysis of the critical head difference model developed by Sellmeijer. This model is applicable to the piping failure mechanism, which addresses backward internal erosion beneath dikes with predominantly horizontal seepage paths.

### Define model

First, let's import the necessary packages:

In [1]:
from streams import *
import numpy as np

The critical head difference, $H_c$, according to the Sellmeijer's model is described by the following equations:

$F_{resistance}=\eta\cdot \frac{\gamma_{sub,particles}}{\gamma_{water}}\cdot \tan \theta_{sellmeijer,rev}$


$F_{scale}=\frac{d_{70.m}}{\sqrt[3]{\kappa\cdot L}}\cdot\left(\frac{d_{70}}{d_{70.m}}\right)^{0.4}$ and $\kappa = \frac{\nu_{water}}{g}\cdot k$


$F_{geometry}=0.91\cdot \left(\frac{D}{L}\right)^{\frac{0.28}{\left(\frac{D}{L}\right)^{2.8}-1}+0.04}$

$H_c = F_{resistance} \cdot F_{scale} \cdot F_{geometry} \cdot L$

where: <br>
$L$ - seepage length (m) <br>
$D$ - thickness of upper sand layer (m) <br>
$\theta$ - bedding angle ($\circ$) <br>
$d_{70}$ - particle diameter (m) <br>
$k$ - permeability of the upper sand layer (m/s)

In [2]:
def model_sellmeijer(k, L, d70, D):

    RD = 0.725
    RDm = 0.725
    d70m = 2.08e-4
    nu = 1.33e-6
    eta = 0.25
    theta = 37.0

    F_resistance = 1.65*eta*np.tan(theta/180*np.pi)*(RD/RDm)**0.35
    F_scale = d70m/(nu/9.81*k*L)**(1/3)*(d70/d70m)**0.39
    F_geometry = 0.91*(D/L)**(0.28/(((D/L)**2.8)-1)+0.04)

    if D==L:
        F_geometry = 1.0
    
    delta_h_c = F_resistance * F_scale * F_geometry * L

    return delta_h_c

### Perform sensitivity analysis

The goal is to estimate the effect of the input parameters $k$, $L$, $d70$ and $D$ on the critical head difference. 
To achieve this, we conduct a sensitivity analysis, we begin by creating a sensitivity project and defining the model:

In [3]:
project = SensitivityProject()
project.model = model_sellmeijer

We define all the input parameters of the model as deterministic variables:

In [4]:
project.variables["k"].distribution = "deterministic"
project.variables["k"].mean = 0.000245598

project.variables["L"].distribution = "deterministic"
project.variables["L"].mean = 40.0

project.variables["d70"].distribution = "deterministic"
project.variables["d70"].mean = 0.00019

project.variables["D"].distribution = "deterministic"
project.variables["D"].mean = 30.0

The following code estimates the effect of each parameter on the critical head difference. Each parameter is assumed to follow a log-normal distribution, with the mean value as specified above and a standard deviation equal to 25% of the mean.

We use the `crude_monte_carlo` method and define the relevant settings: `minimum_samples` and `maximum_samples`.

The sensitivity analysis is performed using `project.run()`, and the results can be accessed from `project.stochast`.

In [13]:
project.settings.sensitivity_method = "crude_monte_carlo"
project.settings.minimum_samples = 100
project.settings.maximum_samples = 200

for var in ["k", "L", "d70", "D"]:
        
        project.variables[var].distribution = "log_normal"
        project.variables[var].deviation = project.variables[var].mean*0.25
    
        project.run()
        sens = project.stochast
        
        print(f"Effect of {var}:( mean = {sens.mean}, std = {sens.deviation}, median = {sens.get_quantile(0.5)} )")
        
        project.variables[var].distribution = "deterministic"


Effect of k:( mean = 2.4019276265421907, sdt = 0.19453479477860572, median = 2.3877313179773196 )
Effect of L:( mean = 2.321004963350936, sdt = 0.4642433287202371, median = 2.262600485952351 )
Effect of d70:( mean = 2.329505145238177, sdt = 0.23575661470502998, median = 2.29532402478645 )
Effect of D:( mean = 2.384997868315509, sdt = 0.08490030120560915, median = 2.3692996588643958 )


The results of the sensitivity analysis are, by default, presented in a histogram, which can be accessed as follows:

In [17]:
for ii in range(0, len(sens.histogram_values)):

    print(f"bin = ({sens.histogram_values[ii].lower_bound}, {sens.histogram_values[ii].upper_bound}), amount = {sens.histogram_values[ii].amount}")

bin = (2.219438060201332, 2.226061777269313), amount = 1.0
bin = (2.226061777269313, 2.232685494337294), amount = 1.0
bin = (2.239309211405274, 2.245932928473255), amount = 1.0
bin = (2.245932928473255, 2.252556645541236), amount = 2.0
bin = (2.252556645541236, 2.2591803626092166), amount = 1.0
bin = (2.265804079677197, 2.272427796745178), amount = 1.0
bin = (2.272427796745178, 2.2790515138131586), amount = 3.0
bin = (2.2790515138131586, 2.2856752308811394), amount = 6.0
bin = (2.2856752308811394, 2.29229894794912), amount = 8.0
bin = (2.29229894794912, 2.2989226650171006), amount = 5.0
bin = (2.3055463820850814, 2.3121700991530623), amount = 8.0
bin = (2.3121700991530623, 2.318793816221043), amount = 7.0
bin = (2.318793816221043, 2.3254175332890235), amount = 7.0
bin = (2.3254175332890235, 2.3320412503570043), amount = 6.0
bin = (2.3320412503570043, 2.338664967424985), amount = 7.0
bin = (2.338664967424985, 2.3452886844929655), amount = 4.0
bin = (2.3452886844929655, 2.351912401560946